In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.options.display.max_columns = 100
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor as RF
from sklearn.metrics import r2_score as r2
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

In [2]:
excel_file = 'F:\\PfDS\\train.csv'
data_base = pd.read_csv(excel_file)
data_base.reset_index();

# Train data fix

In [3]:
data_base.loc[data_base.HouseYear == 4968, 'HouseYear'] = 1968

data_base.loc[data_base.HouseYear == 20052011, 'HouseYear'] = 2008

data_base.loc[data_base.Id == 5927, 'Rooms'] = 2

data_base.loc[data_base.Id == 8491, 'Rooms'] = 1

data_base.loc[data_base.Id == 14003, 'Rooms'] = 2

data_base.loc[data_base.Id == 14865, 'Rooms'] = 2

data_base.loc[data_base.Id == 4282, 'Rooms'] = 4

data_base.loc[data_base.Id == 6358, 'Rooms'] = 1

data_base.loc[data_base.Id == 7594, 'Rooms'] = 1

data_base.loc[data_base.Id == 238, 'Rooms'] = 4

data_base.loc[data_base.Id == 4214, 'Rooms'] = 1

data_base.loc[(data_base.Square < 25) & (data_base.Rooms == 1), ['Square', 'LifeSquare', 'KitchenSquare']] = [41.35, 9.76, 25.10]

In [4]:
data_base = data_base.drop('Healthcare_1', axis = 1)

data_base = data_base.drop(data_base[data_base.Rooms == 0].index)

data_base = data_base.drop(data_base[data_base.Square < 20].index)

data_base = data_base.drop(data_base[data_base.Square > 300].index)

data_base = data_base.drop(data_base[data_base.Id == 16772].index)

data_base = data_base.drop(data_base[data_base.Id == 13772].index)

data_base = data_base.drop(data_base[data_base.Id == 3052].index)

data_base = data_base.drop(data_base[(data_base.Price > 500000) & (data_base.Square < 85) & (data_base.Rooms < 3)].index)

data_base.KitchenSquare.replace([0, 1, 2], np.NaN, inplace= True)

mean_price = data_base.groupby('Rooms', as_index = False)[['Price']].mean().rename(columns = {'Price': 'mean_price'})

mean_square = data_base.groupby('Rooms', as_index = False)[['Square']].mean().rename(columns={'Square':'mean_square'})

kitchen_square_mean = data_base.groupby('Rooms', as_index = False)[['KitchenSquare']].mean().rename(columns = {'KitchenSquare':'meanKitchenSquare'})

life_square_mean = data_base.groupby('Rooms', as_index = False)[['LifeSquare']].mean().rename(columns = {'LifeSquare':'meanLifeSquare'})

mean_data = pd.merge(mean_price, mean_square, on = 'Rooms')

mean_data = pd.merge(mean_data, kitchen_square_mean, on = 'Rooms')

mean_data = pd.merge(mean_data, life_square_mean, on = 'Rooms')

mean_data.mean_price = mean_data.mean_price.astype(int)

data_base = pd.merge(data_base, mean_data, on = 'Rooms',  how = 'left')

data_base.Ecology_3 = (data_base.Ecology_3 == 'A').astype(int)

data_base.Ecology_2 = (data_base.Ecology_2 == 'A').astype(int)

data_base.Shops_2 = (data_base.Shops_2 == 'A').astype(int)

data_base.KitchenSquare = data_base.KitchenSquare.combine_first(data_base.meanKitchenSquare)

Id_Kitchen = data_base.loc[data_base.Square < data_base.KitchenSquare, ['Id','Price', 'Rooms', 'Square']]

data_base.loc[data_base.Id.isin(Id_Kitchen.Id), 'KitchenSquare'] = data_base.meanKitchenSquare

data_base.LifeSquare = data_base.LifeSquare.combine_first(data_base.meanLifeSquare)

data_base.loc[data_base.Id.isin(data_base.loc[data_base.Square < data_base.LifeSquare, 'Id']), 'LifeSquare'] = data_base.meanLifeSquare

big_square_id = data_base.loc[(data_base['Rooms'] == 5) & (data_base['Square'] < 100), 'Id']

data_base.loc[data_base.Square < data_base.LifeSquare, 'LifeSquare'] = data_base.Square - data_base.KitchenSquare 

data_base.loc[data_base.KitchenSquare > 25, 'KitchenSquare'] = data_base.meanKitchenSquare

data_base.loc[data_base.HouseFloor < 1, 'HouseFloor'] = data_base['Floor']

data_base.loc[data_base.HouseFloor > 50, 'HouseFloor'] = data_base.Floor

floors = data_base.loc[data_base.HouseFloor > 3, ['HouseFloor', 'HouseYear']]

mean_House_Floor = floors.groupby('HouseYear', as_index = False)['HouseFloor'].mean().rename(columns = {'HouseFloor':'mean_House_Floor'})

mean_House_Floor = mean_House_Floor.astype(int)

data_base = pd.merge(data_base, mean_House_Floor, on = 'HouseYear', how = 'left')

data_base.loc[data_base.Floor > data_base.HouseFloor, 'HouseFloor'] = data_base.mean_House_Floor

data_base.loc[data_base.Floor > data_base.HouseFloor, 'HouseFloor'] = data_base.Floor

data_base.loc[data_base.LifeSquare < data_base.KitchenSquare, 'LifeSquare'] = data_base.Square - data_base.KitchenSquare

data_stat1 = data_base.groupby(['DistrictId', 'Rooms'], as_index=False)[['Price']].mean().rename(columns={'Price': 'district_mean_price'})

data_base = pd.merge(data_base, data_stat1, on=['DistrictId', 'Rooms'], how='left')


# Test data fix

In [5]:
excel_file2 = 'F:\\PfDS\\test.csv'
data = pd.read_csv(excel_file2)
data.reset_index();

In [6]:
data.loc[(data.Square < 25) & (data.Rooms == 1), ['Square', 'LifeSquare', 'KitchenSquare']] = [41.35, 9.76, 25.10]
data = data.drop('Healthcare_1', axis = 1)
data.loc[data.HouseYear > 2018, 'HouseYear'] = 2018
data.loc[data.Id == 14498, 'Rooms'] = 4
data.loc[data.Id == 14498, 'KitchenSquare'] = 14
data.loc[data.Id == 14498, 'LifeSquare'] = 149
data.loc[data.Id == 14498, 'HouseFloor'] = 10
data.loc[data.Id == 170, 'Square'] = 63
data.loc[data.Id == 11470, 'Rooms'] = 1
data.loc[data.Id == 1435, 'Rooms'] = 2

In [7]:
data.KitchenSquare.replace([0, 1, 2], np.NaN, inplace= True)

mean_square = data.groupby('Rooms', as_index = False)[['Square']].mean().rename(columns={'Square':'mean_square'})

kitchen_square_mean = data.groupby('Rooms', as_index = False)[['KitchenSquare']].mean().rename(columns = {'KitchenSquare':'meanKitchenSquare'})

life_square_mean = data.groupby('Rooms', as_index = False)[['LifeSquare']].mean().rename(columns = {'LifeSquare':'meanLifeSquare'})

mean_data2 = pd.merge(kitchen_square_mean, mean_square, on = 'Rooms')

mean_data2 = pd.merge(mean_data2, life_square_mean, on = 'Rooms')

data = pd.merge(data, mean_data2, on = 'Rooms',  how = 'left')

data.Ecology_3 = (data.Ecology_3 == 'A').astype(int)

data.Ecology_2 = (data.Ecology_2 == 'A').astype(int)

data.Shops_2 = (data.Shops_2 == 'A').astype(int)

data.KitchenSquare = data.KitchenSquare.combine_first(data.meanKitchenSquare)

Id_Kitchen = data.loc[data.Square < data.KitchenSquare, 'Id']

data.loc[data.Id.isin(Id_Kitchen), 'KitchenSquare'] = data.meanKitchenSquare

data.LifeSquare = data.LifeSquare.combine_first(data.meanLifeSquare)

data.loc[data.Id.isin(data.loc[data.Square < data.LifeSquare, 'Id']), 'LifeSquare'] = data.meanLifeSquare

big_square_id = data.loc[(data['Rooms'] == 5) & (data['Square'] < 100), 'Id']

data.loc[data.Square < data.LifeSquare, 'LifeSquare'] = data.Square - data.KitchenSquare 

data.loc[data.KitchenSquare > 25, 'KitchenSquare'] = data.meanKitchenSquare

data.loc[data.HouseFloor < 1, 'HouseFloor'] = data['Floor']

data.loc[data.HouseFloor > 50, 'HouseFloor'] = data.Floor

floors = data.loc[data.HouseFloor > 3, ['HouseFloor', 'HouseYear']]

mean_House_Floor = floors.groupby('HouseYear', as_index = False)['HouseFloor'].mean().rename(columns = {'HouseFloor':'mean_House_Floor'})

mean_House_Floor = mean_House_Floor.astype(int)

data = pd.merge(data, mean_House_Floor, on = 'HouseYear', how = 'left')

data.loc[data.Floor > data.HouseFloor, 'HouseFloor'] = data.mean_House_Floor

data.loc[data.Floor > data.HouseFloor, 'HouseFloor'] = data.Floor

data.loc[data.LifeSquare < data.KitchenSquare, 'LifeSquare'] = data.Square - data.KitchenSquare

data = pd.merge(data, data_stat1, on=['DistrictId', 'Rooms'], how='left')
data = pd.merge(data, mean_price, on = 'Rooms', how='left')

data.loc[data.district_mean_price.isna(), 'district_mean_price'] = data.Square * (data_base.Price / data_base.Square).mean() 

data.loc[data.mean_price.isna(), 'mean_price'] = data.Square * (data_base.Price / data_base.Square).mean() 

# Training

In [8]:
X = data_base.drop(columns = 'Price')
y = data_base['Price']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [10]:
parameters = [{'n_estimators': np.arange(20, 30), 
'max_features': np.arange(15, 20),
'max_depth': np.arange(5, 9)}]

In [11]:
clf = GridSearchCV(estimator=RandomForestRegressor(random_state=100),
                  param_grid=parameters,
                  scoring='r2',
                  cv=5)

In [12]:
clf.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=100, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_estimators': array([20, 21, 22, 23, 24, 25, 26, 27, 28, 29]), 'max_features': array([15, 16, 17, 18, 19]), 'max_depth': array([5, 6, 7, 8])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=0)

In [13]:
clf.best_params_

{'max_depth': 8, 'max_features': 15, 'n_estimators': 28}

In [10]:
clf = RandomForestRegressor(max_depth = 8, max_features = 15, n_estimators = 28, random_state = 100)

In [11]:
clf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=8,
           max_features=15, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=28, n_jobs=None, oob_score=False, random_state=100,
           verbose=0, warm_start=False)

In [12]:
data_base_pred = clf.predict(X_test)

In [14]:
r2(y_test, data_base_pred)

0.7617553173027127

In [16]:
final_pred = clf.predict(data)

In [20]:
final_pred = pd.DataFrame(final_pred)

In [27]:
final = final_pred.join(data.Id)

In [34]:
final = final.rename(columns={0:'Price'})

In [38]:
final = final[['Id', 'Price']]

In [41]:
final.loc[:, ['Id', 'Price']].to_csv('Margulis_predictions.csv', index=None)